# Import packages

In [ ]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import youdao

# [Mesh]词的搜索结果,筛选Pubtator结果

In [ ]:
keyword = json.load(open("../1.3.KeyWord/keyword.json", "r"))
file_dir = "Bactology"

input_path = "../1.2.PubTator/output/%s/" % file_dir
sub = [
    "BACKGROUND:", "BACKGROUND/AIM:", "METHOD:", "METHODS:", "FINGDING",
    "FINDINGS:", "RESULT", "RESULTS:", "CONCLUSION", "CONCLUSIONS:",
    "OBJECTIVE:", "OBJECTITVES"
]

# 建立物种大类列表
sp_l = [ele for ele in os.listdir(input_path) if "_done" not in ele]
sp_l = [ele for ele in sp_l if ".ipynb" not in ele]

b = []
paper_ad = []
for sp in sp_l:

    paper_d = {} #所有物种列表
    paper_pd = {} #所有物种的paper列表

    paper_ld = {} # 所有小于10的物种列表
    paper_md = {} # 所有大于10的物种列表
    paper_pld = {} # 所有小于10的物种的paper列表
    paper_pmd = {} # 所有大于10的物种的paper列表
    
    youdao.mk_dir("%s/%s" % (file_dir, sp))

    # 建立这个大类中物种列表
    print(sp)
    spec_l = [
        ele for ele in os.listdir("%s/%s" % (input_path, sp))
        if ".ipynb" not in ele
    ]

    for spec in spec_l:

        spec_d = {}  # 用于储存paper
        spec_pd = {}
        file_l = [
            ele for ele in os.listdir("%s/%s/%s" % (input_path, sp, spec))
            if ".json" in ele
        ]  # 建立这个物种的三种搜索结果json文件列表

        for file in file_l:

            # 读取json文件中的paper并建立列表
            input_name = "%s/%s/%s/%s" % (input_path, sp, spec, file)
            data_l = json.load(open("%s" % input_name, 'r'))

            for paper in data_l:

                # 读取pubtator的注释
                title = paper["passages"][0]["text"]
                ab = paper["passages"][1]["text"]
                screen_l = keyword["LungRelated"]
                key_l = [
                    word for word in keyword["LungRelated"]
                    if word in title + ab
                ]

                if len(key_l) > 0:  # 筛选3

                    full = "SPECIES:%s\nID:%s\nTITLE:%s\nABSTRACT:%s" % (
                        spec.replace("_", " "), paper["id"], title, ab)

                    # read title annotation
                    title_anno = paper["passages"][0]["annotations"]
                    title_disease_l = [
                        i['text'] for i in title_anno
                        if i['infons']['type'] == 'Disease'
                    ]
                    title_species_l = [
                        i['text'] for i in title_anno
                        if i['infons']['type'] == 'Species'
                    ]

                    ab_anno = paper["passages"][1]["annotations"]
                    full_disease_l = title_disease_l + [
                        i['text']
                        for i in ab_anno if i['infons']['type'] == 'Disease'
                    ]
                    full_species_l = title_species_l + [
                        i['text']
                        for i in ab_anno if i['infons']['type'] == 'Species'
                    ]
                    pubtator_d = {}

                    pubtator_d["dis"] = title_disease_l + full_disease_l
                    #                     pubtator_d["spec"] = title_species_l + full_species_l
                    pubtator_d["spec"] = spec.replace("_", " ")

                    for word in sub:
                        if word in full:
                            full = full.replace(word, "\n%s" % word)
                            full = full.replace("\n\n", "\n")

                    spec_d[paper["id"]] = full
                    spec_pd[paper["id"]] = pubtator_d

        paper_d[spec] = spec_d
        paper_pd[spec] = spec_pd
    paper_ad.append(paper_d)

    a = []
    # for specc in paper_d:
    #     if len(paper_d[specc]) >= 10:
    #         if len(paper_d[specc]) > 0:
    #             for paperr in paper_d[specc]:
    #                 a.append([int(paperr), specc.replace("_", " ")])
    #                 with open(
    #                         "%s/%s/%s.txt" %
    #                     ("paper_ld", sp, specc + "_" + paperr), "w") as f:
    #                     f.write(paper_d[specc][paperr])
    for specc in paper_d:
        if len(paper_d[specc]) > 0:
            for paperr in paper_d[specc]:
                a.append([int(paperr), specc.replace("_", " ")])
                # with open(
                #         "%s/%s/%s.txt" %
                    # (file_dir, sp, specc + "_" + paperr), "w",  encoding="utf-8") as f:
                    # f.write(paper_d[specc][paperr])